<a href="https://colab.research.google.com/github/keshav434/Sentiment-Analysis/blob/main/Sentimental_Part_1_and_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Point No. 1
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import re
from sklearn.model_selection import train_test_split
#from cleantext import clean
import numpy as np

#Point No. 2
data = pd.read_csv('/content/drive/My Drive/deforestation_sentiment_train.csv')
print("1.  Details after data loading")
#print(data.describe)
print(data.columns)
print(data.head(5))
print(data.tail(5))

#Point No. 3
#def clean_text(sentence):
#Removes all special characters from sentence. It will also strip out
# extra whitespace and makes the string lowercase.
#  clean(sentence, no_emoji=True)
#  return re.sub(r'[\\\\/:*«`\'?¿";!<>,.|]', '', sentence.lower().strip())

def clean_text(sentence):
    # Remove URLs
     sentence = re.sub(r'http\S+', '', sentence)
    # Remove hashtags and mentions
     sentence = re.sub(r'#\w+|\@\w+', '', sentence)
    # Remove special characters and numbers
     sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
    # Convert to lowercase
     sentence = sentence.lower()
    # Remove extra spaces
     sentence = re.sub(r'\s+', ' ', sentence).strip()
    # Remove emojis
#   clean(sentence, no_emoji=True)
     return sentence

#data["clean_text"] = data["text"]
data['clean_text'] = data['text'].apply(clean_text)
print("2. Clean Text -----------")
print(data['clean_text'])
print(data.columns)
print(data.head(5))
#Point # 4
X = data["clean_text"]
y = data["sentiments_list"]  # sentiment label
# y = ['', "{'negative'}", "{'neutral', 'negative'}" .. ]
print("X,y",X, y)
X_updated = []
y_updated = []
print("len(y)->",len(y))
for sentiment_list_ind in range(0, len(y)):
    #print(str(y[sentiment_list_ind]))
    if (str(y[sentiment_list_ind]) == 'nan'):
        continue
    else:
        # print(y[sentiment_list_ind])
        current_sentiment_list = eval('[' + y[sentiment_list_ind][1 : -1] + ']')
        neg_count = 0
        neutral_count = 0
        pos_count = 0
        #print("current_sentiment_list", current_sentiment_list)
        for sentiment in current_sentiment_list:
            if (sentiment == 'positive'):
                pos_count += 1
            if (sentiment == 'negative'):
                neg_count += 1
            if (sentiment == 'neutral'):
                neutral_count += 1
        # neg = 2, neutral = 3, pos = 4
        score = (neg_count * 0 + neutral_count * 0.5 + pos_count * 1.0)/(neg_count + neutral_count + pos_count)

        y_updated.append(score)
        X_updated.append(X[sentiment_list_ind])

for i in range(0, len(X_updated)):
    print(X_updated[i][:30] + " ... ", y_updated[i])
print("   ")

# Point No. 5

# labels = set(y)

# Split all our sentences
elements = (' '.join([sentence for sentence in X_updated])).split()
elements.append("<UNK>")
print(len(elements))
print(len(np.unique(elements)))
print(elements[:20])



#Training a Recurrent Neural Network:
#   Collect a vocabulary of words
#   Generate look-up tables, word-to-int and int-to-word
#   Represent my data as integers not strings
#   One-hot encode them

def create_lookup_tables(text):
#Create lookup tables for vocabulary  :param text: The text split into words
#return: A tuple of dicts (vocab_to_int, int_to_vocab)
    vocab = set(text)

    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = {v:k for k, v in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab


#it creates two look-up tables…
vocab_to_int, int_to_vocab = create_lookup_tables(elements)
#print("vocab_to_int", vocab_to_int)
#print("int_to_vocab", int_to_vocab)
print("***************vocab_to_int.keys, vocab_to_int.values ********")
print(list(vocab_to_int.keys())[:5], list(vocab_to_int.values())[:5])
print("***************int_to_vocab.keys, int_to_vocab.values ********")
print(list(int_to_vocab.keys())[:5], list(int_to_vocab.values())[:5])
#languages_to_int, int_to_languages = create_lookup_tables(y)

#size of our vocabulary.
print("Vocabulary of our dataset: {}".format(len(vocab_to_int)))

#Point No. 6
#Now is also a good time to split our data into a train and test set.
#
# Python’s sklearn package provides us with functionality to do this, using the
#code below. Note that X_train will be the training data (i.e. review texts) used during the training
#phase, while y_train will be the labels (positive, negative, neutral) used during training. X_test and y_test
#are held out at this stage so we can use them for evaluation later on.

pos_count = 0
pos_indices = []
neu_count = 0
neu_indices = []
neg_count = 0
neg_indices = []
for i in range(0, len(y_updated)):
    if (y_updated[i] > 0.6667):
        pos_count += 1
        pos_indices.append(i)
    elif (y_updated[i] > 0.3334):
        neu_count += 1
        neu_indices.append(i)
    else:
        neg_count += 1
        neg_indices.append(i)
print("Data stats - ")
print(pos_count, neu_count, neg_count)

X_test = []
y_test = []
for category in [pos_indices, neu_indices, neg_indices]:
    for i in range(0, 4):
        X_test.append(X_updated[category[i]])
        y_test.append(y_updated[category[i]])
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = []
y_train = []
for category in [pos_indices, neu_indices, neg_indices]:
    for i in range(4, len(category)):
        X_train.append(X_updated[category[i]])
        y_train.append(y_updated[category[i]])
X_train = np.array(X_train)
y_train = np.array(y_train)

print("Feature Shapes:")
print("\tTrain set: \t\t{}".format(X_train.shape), "\n\tTest set: \t\t{}".format(X_test.shape))
print("Totals:\n\tWords in our Dataset: {}\n".format(len(np.unique(elements))))

#3. Represent data numerically…

def convert_to_int(data, data_int):
    #Converts all our text to integers :param data: The text to be converted
    #:return: All sentences in ints
    all_items = []
    for sentence in data:
        all_items.append([data_int[word] if word in data_int else data_int["<UNK>"] for word in sentence.split()])
    return all_items


# Convert our inputs
X_test_encoded = convert_to_int(X_test, vocab_to_int)
X_train_encoded = convert_to_int(X_train, vocab_to_int)

print(X_train_encoded)
print(X_test_encoded)

#
print(y_train)
print(y_test)

def generate_encoding(sample, vocab_to_int):
    sorted_words = np.array(list(vocab_to_int.keys()))
    embedding = np.zeros(len(sorted_words))
    for word_ind in sample:
            embedding[word_ind] += 1
    return embedding


X_train_encoded_embedding = []
print(len(X_train_encoded))
for encoded_val in X_train_encoded:
        X_train_encoded_embedding.append(generate_encoding(encoded_val, vocab_to_int))
X_train_encoded_embedding = np.array(X_train_encoded_embedding)
print(X_train_encoded_embedding.shape)
y_train = np.array(y_train)
print(y_train.shape)

X_test_encoded_embedding = []
print(len(X_test_encoded))
for encoded_val in X_test_encoded:
        X_test_encoded_embedding.append(generate_encoding(encoded_val, vocab_to_int))
X_test_encoded_embedding = np.array(X_test_encoded_embedding)
print(X_test_encoded_embedding.shape)
y_test = np.array(y_test)
print(y_test.shape)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
#Point No 7
# Define the model
model = Sequential()

# Add the layers
model.add(Dense(128, activation='relu', input_shape=(1768,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
optimizer = tf.keras.optimizers.SGD(
    learning_rate=1e-4
    )
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

# Print the model summary
print(model.summary())

#

# stats
pos_count = 0
pos_indices = []
neu_count = 0
neu_indices = []
neg_count = 0
neg_indices = []
for i in range(0, len(y_train)):
    if (y_train[i] > 0.6667):
        pos_count += 1
        pos_indices.append(i)
    elif (y_train[i] > 0.3334):
        neu_count += 1
        neu_indices.append(i)
    else:
        neg_count += 1
        neg_indices.append(i)
print(pos_count, neu_count, neg_count)
# prints (35, 106, 14)
# creating validation set with 4 from each
X_val_encoded_embedding = []
y_val = []
for category in [pos_indices, neu_indices, neg_indices]:
    for i in range(0, 4):
        X_val_encoded_embedding.append(X_train_encoded_embedding[category[i]])
        y_val.append(y_train[category[i]])
X_val_encoded_embedding = np.array(X_val_encoded_embedding)
y_val = np.array(y_val)
# augmenting dataset
# X_train_encoded_embedding, y_train
import random

X_train_encoded_embedding_augmented = []
y_train_augmented = []

for category in [pos_indices[4 : ], neu_indices[4 : ], neg_indices[4 : ]]:
    for i in range(30):
        number_of_tweets_to_append = random.randint(0, 3) + 1
        indices_of_tweets_to_append = []
        for j in range(0, number_of_tweets_to_append):
            indices_of_tweets_to_append.append(category[random.randint(1, len(category) - 1)])
        augmented_score = 0
        for j in range(0, number_of_tweets_to_append):
            augmented_score += y_train[indices_of_tweets_to_append[j]]
        augmented_score = augmented_score * 1.0/number_of_tweets_to_append
        augmented_sample = np.zeros(1768)
        for j in range(0, number_of_tweets_to_append):
            augmented_sample = augmented_sample + X_train_encoded_embedding[indices_of_tweets_to_append[j]]
        # print(number_of_tweets_to_append, indices_of_tweets_to_append, augmented_score)
        X_train_encoded_embedding_augmented.append(augmented_sample)
        y_train_augmented.append(augmented_score)

X_train_encoded_embedding_augmented = np.array(X_train_encoded_embedding_augmented)
y_train_augmented = np.array(y_train_augmented)

print("Augmented Dataset X : ", X_train_encoded_embedding_augmented.shape)
print("Augmented Dataset y : ", y_train_augmented.shape)
print("Augmented Dataset X[75] : ", X_train_encoded_embedding_augmented[75])
print("Augmented Dataset y[75] : ", y_train_augmented[75])

print("Average y_train : ", np.mean(y_train))
print("Average y_train_augmented : ", np.mean(y_train_augmented))

train_history_augmented = model.fit(X_train_encoded_embedding_augmented, y_train_augmented, epochs=10, batch_size=2, validation_data = (X_val_encoded_embedding, y_val)
          , verbose = 1
        )

y_pred = model.predict(X_test_encoded_embedding, verbose = 0)

test_mae = 0
test_mse = 0
# mean absolute error, mean squared error, crossentropy (?)

for i in range(0, len(y_pred)):
    ae = abs(y_pred[i] - y_test[i])
    se = ae * ae
    test_mae += ae
    test_mse += se
test_mae = test_mae/len(y_pred)
test_mse = test_mse/len(y_pred)
print("For Augmented Model - ")
print("Mean Absolute Error : ", round(test_mae[0], 4), "Mean Squared Error : ", round(test_mse[0], 4))
print("*************************End of Part 1************")
# 0 -> negative, 0.25 -> neutral negative, 0.5 -> neutral, 0.75 -> neutral_positive, 1 -> positive

count_right_pred = 0
for i in range(0, len(y_pred)):
    # positive, neutral, negative
    if (y_pred[i] > 0.6667):
        pred_sentiment = 1.0
    elif (y_pred[i] > 0.3334):
        pred_sentiment = 0.5
    else:
        pred_sentiment = 0.0
    if (y_test[i] > 0.6667):
        true_sentiment = 1.0
    elif (y_test[i] > 0.3334):
        true_sentiment = 0.5
    else:
        true_sentiment = 0.0
    if (pred_sentiment == true_sentiment):
        print("Right Classification")
        print("Sentence -", X_test[i])
        print("Predicted :", y_pred[i][0], "True", y_test[i])
        print(" ")
        count_right_pred += 1
    else:
        print("Wrong Classification")
        print("Sentence -", X_test[i])
        print("Predicted :", y_pred[i][0], "True", y_test[i])
        print(" ")
print("Accuracy of Model : ", round(count_right_pred/len(y_pred), 2))
print("Got", count_right_pred, "out of", len(y_test), "correct.")


#######################Part2 - DistilBirt for sentiment analysis##########################
#Point No. 1
'''import pandas as pd
import re
from sklearn.model_selection import train_test_split
from cleantext import clean
import numpy as np

#Point No. 2
data = pd.read_csv('deforestation_sentiment_train.csv')
print("1.  Details after data loading")
#print(data.describe)
print(data.columns)
print(data.head(5))
print(data.tail(5))

#Point No. 3

#def clean_text(sentence):
#Removes all special characters from sentence. It will also strip out
# extra whitespace and makes the string lowercase.
#  clean(sentence, no_emoji=True)
#  return re.sub(r'[\\\\/:*«`\'?¿";!<>,.|]', '', sentence.lower().strip())



def clean_text(sentence):
    # Removes all special characters from sentence.
    # It will also strip out extra whitespace and makes the string lowercase.

    # Remove URLs
    sentence = re.sub(r'http\S+', '', sentence)
    # Remove hashtags and mentions
    sentence = re.sub(r'#\w+|\@\w+', '', sentence)
    # Remove special characters and numbers
    sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    # Remove emojis
    clean(sentence, no_emoji=True)
    return sentence

data['clean_text'] = data['text'].apply(clean_text)
print("2. Clean Text -----------")
print(data['clean_text'])
print(data.columns)
print(data.head(5))'''
#Point # 4
X = data["clean_text"]
y = data["sentiments_list"]  # sentiment label
# y = ['', "{'negative'}", "{'neutral', 'negative'}" .. ]
print("X,y",X, y)
X_updated = []
y_updated = []
print("len(y)->",len(y))
for sentiment_list_ind in range(0, len(y)):
    if (str(y[sentiment_list_ind]) == 'nan'):
        continue
    else:
        current_sentiment_list = eval('[' + y[sentiment_list_ind][1 : -1] + ']')
        neg_count = 0
        neutral_count = 0
        pos_count = 0
        score = 1
        #print("current_sentiment_list", current_sentiment_list)
        for sentiment in current_sentiment_list:
            if (sentiment == 'positive'):
                score = 1
            if (sentiment == 'negative'):
                score = 0
        y_updated.append(score)
        X_updated.append(X[sentiment_list_ind])

for i in range(0, len(X_updated)):
    print(X_updated[i][:30] + " ... ", y_updated[i])
print("   ")
# input("Press any key to continue...")
#Now is also a good time to split our data into a train and test set.

import tensorflow as tf
!pip install transformers
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Assuming you have your input data in 'X_train' and corresponding labels in 'y_train'
# Split the data into train and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_updated, y_updated, test_size=0.2, random_state=42)

# Load the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Tokenize the text data and convert to input tensors

def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf')

X_train_tokenized = tokenize_texts(X_train)
X_test_tokenized = tokenize_texts(X_test)

print("X_train_tokenized ", X_train_tokenized)
print("X_val_tokenized = ", X_test_tokenized)
# Create the Keras model
model = Sequential()
model.add(tf.keras.layers.Input(shape=(None,), dtype='int32'))
model.add(distilbert_model)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
print(model.summary())
# input("Model Summary")
print("X_train_tokenized length : ", len(X_train_tokenized))
print("X_train_tokenized keys() : ", X_train_tokenized.keys())
print("X_train_tokenized['input_ids'] length : ", len(X_train_tokenized['input_ids']))
print("X_train_tokenized['attention_mask'] length : ", len(X_train_tokenized['attention_mask']))
print("X_train_tokenized['input_ids'][56] : ", X_train_tokenized['input_ids'][56])
print("X_train_tokenized['attention_mask'][56] : ", X_train_tokenized['attention_mask'][56])
print("y_train length : ", len(y_train))
# Train the model with your data (X_train_tokenized and y_train)
model.fit(np.array(X_train_tokenized['input_ids']), np.array(y_train), epochs=1, batch_size=32, validation_split = 0.2, verbose = 1)

# Evaluate the model on the test set if available
# Assuming you have test data in 'X_test' and corresponding labels in 'y_test'
loss, accuracy = model.evaluate(np.array(X_test_tokenized['input_ids']), np.array(y_test))
print("Test loss:", loss)
print("Test accuracy:", accuracy)

y_pred = model.predict(np.array(X_test_tokenized['input_ids']), verbose = 0)
y_test = np.array(y_test)
X_test = np.array(X_test_tokenized['input_ids'])
print("Length of y_pred : ", len(y_pred))
count_right_pred = 0
for i in range(0, len(y_pred)):
    # negative, neutral, positive
    if (y_pred[i] > 0.5):
        pred_sentiment = 1.0
    else:
        pred_sentiment = 0.0
    if (y_test[i] > 0.5):
        true_sentiment = 1.0
    else:
        true_sentiment = 0.0
    if (pred_sentiment == true_sentiment):
        print("Right Classification")
        print("Sentence -", X_test[i])
        print("Predicted :", y_pred[i][0], "True", y_test[i])
        print(" ")
        count_right_pred += 1
    else:
        print("Wrong Classification")
        print("Sentence -", X_test[i])
        print("Predicted :", y_pred[i][0], "True", y_test[i])
        print(" ")
print("Accuracy of Model : ", round(count_right_pred/len(y_pred), 2))
print("Got", count_right_pred, "out of", len(y_test), "correct.")


Mounted at /content/drive
1.  Details after data loading
Index(['Unnamed: 0', 'id', 'text', 'user_name', 'user_description',
       'user_location', 'user_following', 'user_followers', 'tweets_by_user',
       'user_created_at', 'tweet_created', 'retweets', 'likes', 'hashtags',
       'media', 'search_term', 'emojis', 'sentiments_list',
       'revised_sentiment', 'distillbert_valence'],
      dtype='object')
   Unnamed: 0                   id  \
0      253291  1359216214875262980   
1      253283  1359691122985947137   
2      253280  1359697180987183105   
3      253268  1359718063994667009   
4      253260  1359734007731269632   

                                                text        user_name  \
0  three palm oil firms were responsible for a qu...  hoteldealphuket   
1  despicable #brazil govt using #pandemic as cov...       PhelimKine   
2  🌳🍂🌳impacto humano no #desmatamento / \nhuman i...        isis_0726   
3  deforestation will be the end of the human rac...   AmanzWillie

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

X_train_tokenized  {'input_ids': <tf.Tensor: shape=(155, 56), dtype=int32, numpy=
array([[  101,  3113, 21025, ...,     0,     0,     0],
       [  101,  2095,  2031, ...,     0,     0,     0],
       [  101,  2025,  2069, ...,     0,     0,     0],
       ...,
       [  101,  2035,  2256, ...,     0,     0,     0],
       [  101, 13366, 25794, ...,     0,     0,     0],
       [  101,  1052,  2031, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(155, 56), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}
X_val_tokenized =  {'input_ids': <tf.Tensor: shape=(39, 54), dtype=int32, numpy=
array([[  101,  2057,  2310, ...,     0,     0,     0],
       [  101,  3407, 20657, ...,     0,     0,     0],
       [  101,  1996,  3224, ...,     0,     0,     0],
       ...,
      